## Goals of this part
1. Extend our simple model to the more realistic quantum domain
2. Think about how sparse/dense linear algebra will affect performance
3. Use some Julia plotting functionality to explore the physics

Now we want to add in some quantum features. Essentially what we will do is add some more interactions (super/sub-diagonals) such that the matrix (Hamiltonian) is no longer diagonal in the computation basis. In general this will make finding the lowest energy state very hard! The model we will study is called the *transverse field quantum Ising model*:

$$ \hat{H} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^z \otimes \hat{\sigma}_j^z - h\sum_i \hat{\sigma}_i^x $$

Now, instead of just being *numbers* the $\hat{\sigma}$ are 2x2 *matrices*. I've neglected to write a bunch of outer products with the identity. What's going on is that $\hat{\sigma}_x$ acts as a "flipper", taking up spins to down and vice versa:

$$ \hat{\sigma}^x\left| 1 \right\rangle = \left| 0 \right\rangle $$

and

$$ \hat{\sigma}^x\left| 0 \right\rangle = \left| 1 \right\rangle $$

Clearly, the addition of a bunch of these means our resulting $\hat{H}$ above is *not* diagonal in the simluation basis, at least as long as $h$ is nonzero. (If you need to convince yourself, write out the full 4x4 $\hat{H}$ for a system with just 2 sites.)

$h$ controls how likely it is we will flip a particular site. We will use our simulation to investigate its effects. It's easy to write code to *generate* this matrix:

In [1]:
σᶻ = [1 0; 0 -1]

2×2 Array{Int64,2}:
 1   0
 0  -1

In [2]:
σˣ = [0 1; 1 0]

2×2 Array{Int64,2}:
 0  1
 1  0

In [3]:
σˣ * [1,0]

2-element Array{Int64,1}:
 0
 1

In [4]:
σˣ * [0, 1]

2-element Array{Int64,1}:
 1
 0

In [5]:
kron(σᶻ,σᶻ) # this is σᶻᵢ σᶻⱼ

4×4 Array{Int64,2}:
 1   0   0  0
 0  -1   0  0
 0   0  -1  0
 0   0   0  1

In the formula above, I elided some details. Let's look closer:

$$ \hat{H} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^z \otimes \hat{\sigma}_j^z - h\sum_i \hat{\sigma}_i^x $$

We want to make a matrix that's $2^L \times 2^L$ from a bunch of $4 \times 4$ and $2 \times 2$ matrices. Usually, for compactness, we "suppress" identity matrices and treat them as implicit. Our 4 site Hamiltonian should read:

$$ \hat{H}_4 = -\hat{\sigma}_1^z \hat{\sigma}_2^z \hat{I}_3 \hat{I}_4 - \hat{I}_1 \hat{\sigma}_2^z \hat{\sigma}_3^z \hat{I}_4 - \hat{I}_1 \hat{I}_2 \hat{\sigma}_3^z \hat{\sigma}_4^z - \hat{\sigma}^z_1 \hat{I}_2 \hat{I}_3 \hat{\sigma}_4^z - h\left(\hat{\sigma}_1^x\hat{I}_2 \hat{I}_3\hat{I}_4 + \hat{I}_1 \hat{\sigma}_2^x \hat{I}_3\hat{I}_4 +\hat{I}_1 \hat{I}_2 \hat{\sigma}_3^x\hat{I}_4 + \hat{I}_1 \hat{I}_2 \hat{I}_3 \hat{\sigma}_4^x\right) $$

Now we have a sum of matrices of the right dimension. But what a pain to look at! The short-hand we have above is easier to work with and reason about, but if you get confused, refer back to this. **Note** here that where I've written $ABCD$ I mean $A \otimes B \otimes C \otimes D$ (we'd really run out of room with all those $\otimes$!).

We can see this horrorshow in action:

In [6]:
# An example of a 4 site Hamiltonian

H = -kron(kron(kron(σᶻ,σᶻ), eye(2)), eye(2)) - kron(eye(2), kron(kron(σᶻ,σᶻ), eye(2))) - kron(eye(2), kron(eye(2),kron(σᶻ,σᶻ)))
H -= 2*(kron(kron(kron(σˣ, eye(2)), eye(2)), eye(2)) + kron(eye(2), kron(kron(σˣ, eye(2)), eye(2))) + kron(eye(2), kron(eye(2), kron(σˣ, eye(2)))))

16×16 Array{Float64,2}:
 -3.0  -0.0  -2.0  -0.0  -2.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -1.0  -0.0  -2.0  -0.0  -2.0     -0.0  -0.0  -0.0   0.0  -0.0  -0.0
 -2.0  -0.0   1.0   0.0  -0.0  -0.0     -2.0   0.0  -0.0  -0.0   0.0  -0.0
 -0.0  -2.0   0.0  -1.0  -0.0  -0.0      0.0  -2.0  -0.0  -0.0  -0.0  -0.0
 -2.0  -0.0  -0.0  -0.0   1.0   0.0     -0.0  -0.0  -2.0   0.0  -0.0  -0.0
 -0.0  -2.0  -0.0  -0.0   0.0   3.0  …  -0.0  -0.0   0.0  -2.0  -0.0  -0.0
 -0.0  -0.0  -2.0  -0.0  -2.0   0.0      0.0  -0.0  -0.0  -0.0  -2.0  -0.0
 -0.0  -0.0  -0.0  -2.0   0.0  -2.0     -0.0  -0.0  -0.0  -0.0  -0.0  -2.0
 -2.0  -0.0  -0.0  -0.0  -0.0  -0.0     -2.0   0.0  -2.0  -0.0  -0.0  -0.0
 -0.0  -2.0  -0.0  -0.0  -0.0   0.0      0.0  -2.0  -0.0  -2.0  -0.0  -0.0
 -0.0  -0.0  -2.0   0.0  -0.0  -0.0  …   3.0   0.0  -0.0  -0.0  -2.0  -0.0
 -0.0  -0.0   0.0  -2.0  -0.0  -0.0      0.0   1.0  -0.0  -0.0  -0.0  -2.0
 -0.0  -0.0  -0.0  -0.0  -2.0   0.0     -0.0  -0.0  -1.0   0.0  -2.0  -0.0
 

Now we should think about how to represent the physical spin states. We're going to have `0` or `false` represent "spin up" and `1` or `true` represent "spin down". This choice is 100% arbitrary and it doesn't matter as long as we stay internally consistent.

We can save on storage space by using Julia's native `BitArray` type. We can even build our basis the naive way and convert it at some up-front performance cost. Or, we can build the list of `BitArray`s ourselves:

In [5]:
function bit_rep(element::Integer, L::Integer)
    bit_rep = falses(L)
    for site in 1:L
       bit_rep[site] = (element >> (site - 1)) & 1
    end
    return bit_rep
end

function int_rep(element::BitVector, L::Integer)
    int = 1
    for site in 1:L
       int += (element[site] << (site - 1))
    end
    return int
end

function generate_basis(L::Integer)
    basis = fill(falses(L), 2^L)
    for elem in 1:2^L
        basis[elem] = bit_rep(elem-1, L)
        # elem - 1 because we want 0000000 to correspond to 0, not 1
    end
    return basis
end

function TransverseFieldIsing(L::Integer, h::Real=0.)
    basis = generate_basis(L)
    N = L
    H = zeros(2^N, 2^N)
    bonds = zip(collect(1:N-1), collect(2:N))
    for (index, element) in enumerate(basis)
        # the diagonal part is easy
        diag_term = 0.
        for (site_i, site_j) in bonds
            diag_term -= !xor(element[site_i], element[site_j])
        end
        H[index, index] = diag_term
        # off diagonal part
        for site in 1:N
            new_element = copy(element)
            # flip the bit on the site
            new_element[site] ⊻= true
            new_index = findfirst(basis, new_element)
            H[index, new_index] = -h
        end
    end
    return Hermitian(H), basis
end

TransverseFieldIsing (generic function with 2 methods)

Some things to note:
- This matrix is very sparse! That's pretty generic to quantum systems, and it's a mixed blessing.
- For larger systems, it may make sense to use the Julia sparse matrix types (we'll see how to do this shortly).
- For now we have set $h$ to be the same everywhere. We aren't *forced* to do this - we could introduce *disorder* and see more interesting physics. Something like making $h_i$ dependent on site `i` using `rand(L)`, or even [Distributions.jl](https://github.com/JuliaStats/Distributions.jl). This is a good **exercise** for interested non-physicists.

We can still use Julia's linear algebra methods like `eigfact`, of course, they'll just be slower:

In [8]:
H, basis = TransverseFieldIsing(8, 1.)
@time eigfact(H)

  0.344476 seconds (67.35 k allocations: 5.346 MiB)


Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-11.0084, -11.0084, -9.86622, -9.86622, -9.53473, -9.53473, -9.14576, -9.14576, -8.77229, -8.77229  …  1.77229, 1.77229, 2.14576, 2.14576, 2.53473, 2.53473, 2.86622, 2.86622, 4.00835, 4.00835], [0.0 -0.241289 … 0.0312081 0.0; 0.0 -0.180791 … -0.0390783 0.0; … ; 0.180791 0.0 … 0.0 0.0390783; 0.241289 0.0 … 0.0 -0.0312081])

## Dude, Where's My Parallelism?

For many interesting physical problems (one of which we're about to see) we *don't need* to use multiple nodes anymore. This is good - parallel computing is really cool but it's also really difficult to do well! The promised parallelism is coming! But we should all be happy we can practically look at (some) many-body physics with doing many-body computing (yet).

## It's Just a Phase

Let's vary $h$ and see what happens. Since we're looking at *quantum magnets* we will compute the *overall magnetization*. This quantity is:

$$ M = \frac{1}{N}\sum_{i} \sigma^z_i $$

where $\sigma^z_i$ is the value of the spin on site $i$ when we measure. So, for example:

If $M$ is `0` there is no overall magnetic moment. We divide by the number of sites so that we can compare results for various systems. Since we're using `0` to represent spin down ($\sigma^z = -1$), and `1` to represent spin up ($\sigma^z = +1$), in our code this will look like:

In [9]:
function magnetization(state::Vector, basis)::Float64
    M = 0.
    for (index, element) in enumerate(basis)
        element_M = 0.
        for spin in element
            element_M += (state[index]^2 * (spin ? 1 : -1))/length(element)
        end
        @assert abs(element_M) <= 1
        M += abs(element_M)
    end
    return M
end

magnetization (generic function with 1 method)

Now we would like to examine the effects of $h$. We will:
  1. Find a variety of $h$ to look at.
  2. For each, compute the *lowest energy eigenvector* (groundstate) of the corresponding Hamiltonian.
  3. For each groundstate, compute the overall magnetization $M$.
  4. Plot $M(h)$ for a variety of system sizes, and see if anything cool happens.

In [4]:
using GR
hs = logspace(-2., 2., 10)
Ls = collect(2:10)
for L in Ls
    M = zeros(hs)
    for (i,h) in enumerate(hs)
        H, basis = TransverseFieldIsing(L, h)
        vals, vecs = eig(H)
        groundstate = vecs[:,1]
        M[i] = magnetization(groundstate, basis)
    end
    semilogx(hs, M)
    println(M)
    hold(true)
end
legend(map(x->"L=$x", Ls)...)

LoadError: [91mInterruptException:[39m

## Exercise

So far we have seen functions to compute the energy, and compute and plot the magnetization. There are many other physically interesting quantities we could plot! Try plotting the [magnetic susceptibility](https://en.wikipedia.org/wiki/Magnetic_susceptibility) - how does it vary across the transition?